In [7]:
import arff
import random
########################################################
# Data Loader: train data and test data split
########################################################
train = open('trainProdSelection.arff')

data = []
data.append([(n) for n in train])
train_total = []
for i in range(11,186):
    train_total.append((data[0][i]))
# print (train_total[0])

# print(len(train_data)) # 186
# length with C1 label: 36
# length with C2 label: 26
# length with C3 label: 41
# length with C4 label: 47
# length with C5 label: 36
# print(len(train_model)) # 128
'''
split 70% data randomly from each class individually as experiment data to build model
def data_loader():
return train_model, test_model
'''
train_model, test_model, train_C1, train_C2, train_C3, train_C4, train_C5 =[], [], [], [], [], [], []
for i in range(25):
    train_C1.append(random.choice(train_total[0:35]))
    train_C5.append(random.choice(train_total[-36:-1]))
for i in range(18):
    train_C2.append(random.choice(train_total[36:61]))
for i in range(28):
    train_C3.append(random.choice(train_total[62:102]))
for i in range(32):
    train_C4.append(random.choice(train_total[103:149]))

train_model = train_C1 + train_C2 + train_C3 + train_C4 + train_C5

'''
create the test set that are included in the train_total but train_data
'''
for row in train_total:
    if row not in train_model: test_model.append(row)

print("test data %:", int(len(test_model)/186*100))

# return train_model, test_model


test data %: 44


In [12]:
########################################################
# read the data as comma split
########################################################
# index 1: type
# index 2: lifestyle
# index 3: vacation
# index 4: ecredit
# index 5: salary
# index 6: property
# index 7: label
identity, lifestyle, vacation, ecredit, salary, wealth, label = [], [], [], [], [], [], []

for line in train_model:
    value = line.split(",")
    identity.append(value[0])
    lifestyle.append(value[1])
    vacation.append(value[2])
    ecredit.append(value[3])
    salary.append(value[4])
    wealth.append(value[5])
    label.append(value[6])
# print(identity)

In [125]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  2 13:37:49 2018
@author: yinchenli
"""

class Customer:
    'common class for all types of customer'
    
    def __init__(self, inputString, Train):
        array = inputString.split(',')   
        self.Ctype = array[0]
        self.lifestyle = array[1]
        self.vacation = float(array[2])
        self.eCredit = float(array[3])
        self.salary = float(array[4])
        self.prop = float(array[5].strip())
        self.cla = str(array[-1].strip())
        self.sim = float(0)
        if (Train == True):
            Knn.recordTrainMinMax(2, float(array[2]))
            Knn.recordTrainMinMax(3, float(array[3]))
            Knn.recordTrainMinMax(4, float(array[4]))
            Knn.recordTrainMinMax(5, float(array[5].strip()))
        else:
            Knn.recordTestMinMax(2, float(array[2]))
            Knn.recordTestMinMax(3, float(array[3]))
            Knn.recordTestMinMax(4, float(array[4]))
            Knn.recordTestMinMax(5, float(array[5].strip()))

    def __str__(self):
        return "this customer: type %s, life %s, vac %f eCredit %f salary %f prop %f class %s sim %f" % \
    (self.Ctype, self.lifestyle, self.vacation, self.eCredit, self.salary, self.prop, self.cla, self.sim)
   
        
class Knn:
    'main class to operate all'
    listTrainMin = [None]*6
    listTrainMax = [None]*6
    listTestMin = [None]*6
    listTestMax = [None]*6
    listTrainCustomer = []
    listTestCustomer = []
    
    def readFromFile_Train(name):
        with open(name, "r") as f:         
            for line in f.readlines():
                if not line.startswith("@") and ',' in line:
                    Knn.listTrainCustomer.append(Customer(line, True))
    
    
    def readFromFile_Test(name):
        with open(name, "r") as f:         
            for line in f.readlines():
                if not line.startswith("@") and ',' in line:
                    Knn.listTestCustomer.append(Customer(line, False))
                    
                    
                    
    def initializeTrain():
        i = 3
        for i in range(len(Knn.listTrainMin)):
            Knn.listTrainMin[i]= float('inf')
        for i in range(len(Knn.listTrainMax)):
            Knn.listTrainMax[i] = float('-inf')
            
    def initializeTest():
        i = 3
        for i in range(len(Knn.listTestMin)):
            Knn.listTestMin[i]= float('inf')
        for i in range(len(Knn.listTestMax)):
            Knn.listTestMax[i] = float('-inf')
         
    def recordTrainMinMax(i, value):
        Knn.listTrainMin[i] = min(Knn.listTrainMin[i], value)
        Knn.listTrainMax[i] = max(Knn.listTrainMax[i], value)
    
    def recordTestMinMax(i, value):
        Knn.listTestMin[i] = min(Knn.listTestMin[i], value)
        Knn.listTestMax[i] = max(Knn.listTestMax[i], value)
        
    def normalizeTrain(feaIndex, OriValue):
        normValue = (OriValue - Knn.listTrainMin[feaIndex]) / (Knn.listTrainMax[feaIndex] - Knn.listTrainMin[feaIndex])
        return normValue
    
    def normalizeTest(feaIndex, OriValue):
        normValue = (OriValue - Knn.listTestMin[feaIndex]) / (Knn.listTestMax[feaIndex] - Knn.listTestMin[feaIndex])
        return normValue
               
Knn.initializeTrain()
Knn.readFromFile_Train('trainProdSelection.arff')

Knn.initializeTest()
Knn.readFromFile_Test('testProdSelection.arff')

# normalization
for i in range(len(Knn.listTrainCustomer)):
    t = Knn.listTrainCustomer[i]
#     print('before:')
#     print(t)
    t.eCredit = Knn.normalizeTrain(3, t.eCredit)
    t.vacation = Knn.normalizeTrain(2, t.vacation)
    t.salary = Knn.normalizeTrain(4, t.salary)
    t.prop = Knn.normalizeTrain(5, t.prop)
#     print('after:')
#     print(t)
    
for i in range(len(Knn.listTestCustomer)):
    y = Knn.listTestCustomer[i]
#     print('before:')
#     print(t)
    y.eCredit = Knn.normalizeTrain(3, y.eCredit)
    y.vacation = Knn.normalizeTrain(2, y.vacation)
    y.salary = Knn.normalizeTrain(4, y.salary)
    y.prop = Knn.normalizeTrain(5, y.prop)
#     print('after:')
#     print(t)
    
print(Knn.listTestCustomer[19])

['student', 'spend>saving', '6', '40', '13.62', '3.2804', 'C1\n']
['student', 'spend>saving', '11', '21', '15.32', '2.0232', 'C1\n']
['student', 'spend>saving', '7', '64', '16.55', '3.1202', 'C1\n']
['student', 'spend>saving', '3', '47', '15.71', '3.4022', 'C1\n']
['student', 'spend>saving', '15', '10', '16.96', '2.2825', 'C1\n']
['student', 'spend>saving', '6', '80', '15.5', '3.7338', 'C1\n']
['student', 'spend<saving', '10', '49', '16.86', '5.8639', 'C1\n']
['student', 'spend<saving', '10', '84', '14.66', '3.187', 'C1\n']
['student', 'spend<saving', '9', '74', '13.86', '2.3823', 'C1\n']
['student', 'spend>>saving', '22', '38', '13.88', '0.7394', 'C1\n']
['student', 'spend>>saving', '25', '30', '15.64', '3.1282', 'C1\n']
['student', 'spend>>saving', '31', '15', '14.4', '2.3925', 'C1\n']
['student', 'spend>>saving', '24', '55', '15.55', '1.9857', 'C1\n']
['student', 'spend>>saving', '28', '85', '13.9', '4.3147', 'C1\n']
['student', 'spend<<saving', '25', '24', '15.09', '5.1951', 'C1\n'

In [120]:
########################################################
#  simlarity 
########################################################
# transfer the discrete to simlarity distance: Ctype, lifestyle
w1, w2, w3, w4, w5, w6 = 1,1,1,1,1,1
near = []
# the index of test data
t = 19
print(Knn.listTestCustomer[t])
for j in range(len(Knn.listTrainCustomer)):
# for j in range(3):
    if Knn.listTestCustomer[t].Ctype ==(Knn.listTrainCustomer[j].Ctype):
        sim_type = 1
    else:
        sim_type = 0
        
    if Knn.listTestCustomer[t].lifestyle ==(Knn.listTrainCustomer[j].lifestyle):
        sim_ls = 1
    else:
        sim_ls = 0

# get the overall simlarity distance
    dist_vaca = (Knn.listTrainCustomer[j].vacation - Knn.listTestCustomer[t].vacation)**2
    dis_eCred = (Knn.listTrainCustomer[j].eCredit - Knn.listTestCustomer[t].eCredit)**2
    dist_salary = (Knn.listTrainCustomer[j].salary - Knn.listTestCustomer[t].salary)**2
    dist_prop = (Knn.listTrainCustomer[j].prop - Knn.listTestCustomer[t].prop)**2
    sim_overall = 1/((w1*(1-sim_type) + w2*(1-sim_ls) + w3* dist_vaca+ \
                     w4* dis_eCred+ w5*dist_salary + w6* dist_prop)**0.5)
    a = Knn.listTrainCustomer.copy()
    a[j].sim = sim_overall
#     for row in a:
#         print(row)
    sor = sorted(a, key=lambda Customer:Customer.sim, reverse=False)
    # get the nearest 3 distance
    near = sor[0:3]
#     for row in near:
#         print(row)
#     print(near[0].cla)
    C1, C2, C3, C4, C5 = 0,0,0,0,0
    for n in range(0,3):
        if near[n].cla == 'C1': 
            C1 += 1
        if near[n].cla == 'C2': 
            C2 += 1
        if near[n].cla == 'C3': 
            C3 += 1
        if near[n].cla == 'C4': 
            C4 += 1
        if near[n].cla == 'C5': 
            C5 += 1
#     print(C1, C2, C3, C4, C5)
    
    
print(Knn.listTestCustomer[0])


    


this customer: type engineer, life spend>saving, vac 0.777778 eCredit 0.040698 salary 1.036141 prop 0.068002 class 1.2229 sim 0.000000
this customer: type student, life spend<saving, vac 0.174603 eCredit 0.046512 salary 0.270299 prop 0.210554 class 3.7697 sim 0.000000


In [113]:
len(Knn.listTestCustomer)

21